In [1]:
%cd ..

/misc/vlgscratch4/LakeGroup/wentao/multimodal-baby


In [2]:
import itertools
import functools
from collections import namedtuple
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
from multimodal.multimodal_lit import MultiModalLitModel
from multimodal.multimodal_data_module import PAD_TOKEN_ID, UNK_TOKEN_ID, SOS_TOKEN_ID, EOS_TOKEN_ID
from ngram import NGramModel
from multimodal.utils import map_structure
from analysis_tools.processing import examples_from_dataloader, get_pos_tags, get_model_items, ModelItems, run_model_on_data_batches
from analysis_tools.sumdata import *
from analysis_tools.build_data import build_data
from analysis_tools.pos_tags import *
from analysis_tools.utils import *
from analysis_tools.word_categories import *


figsize = (13, 12)
sns.set_theme(
    style='white',
    font='monospace',
    rc={
        'figure.figsize': figsize,
        'axes.grid': False,
        'xtick.bottom': False,
        'xtick.top': False,
        'ytick.left': False,
        'ytick.right': False,
        'axes.spines.left': False,
        'axes.spines.bottom': False,
        'axes.spines.right': False,
        'axes.spines.top': False,
    }
)

np.set_printoptions(suppress=True, precision=2, linewidth=120)
pd.options.display.width = 120

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pos_mapping = pos_mappings[2][1]

used_poses = ["noun", "verb", "adjective", "adverb", "function word", "cardinal number", "."][:2]

/usr/local/stow/python-3.9/lib/python3.9/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [3]:
def pos_tagged_seq_repr(tokens, pos_tags):
    return ' '.join(f'{token}/{pos}' for token, pos in zip(tokens, pos_tags))


class Template(namedtuple('TemplateTuple', ['seq', 'pos', 'idx'])):
    """A template where the token at idx of seq is to be filled.
    """

    def __str__(self):
        return ' '.join(
            '_' * len(token) if i == self.idx else token
            for i, (token_id, token) in enumerate(
                (token_id, idx2word[token_id]) for token_id in self.seq)
            if token_id not in [SOS_TOKEN_ID, EOS_TOKEN_ID])


def templates_from_example(token_ids, pos_tags):
    for i, (token_id, pos) in enumerate(zip(token_ids, pos_tags)):
        if pos_mapping[pos] in used_poses:
            yield Template(token_ids, pos_tags, i)


def templates_from_examples(examples, pos_tags, print_pos_tagged_seq=False):
    tot, cnt_filtered = 0, 0

    for example, y_pos_tags in zip(examples, pos_tags):
        x, y, y_len, raw_y = example
        tot += 1
        if print_pos_tagged_seq:
            print(pos_tagged_seq_repr(raw_y[0].split(), y_pos_tags[1:]))
        cnt_present_tokens = sum((pos_mapping[pos] not in ["."] for pos in y_pos_tags))
        if cnt_present_tokens <= 2:
            continue
        y_len = y_len.item()
        y = y[:y_len].tolist()
        if UNK_TOKEN_ID in y:
            continue

        cnt_filtered += 1
        yield example, templates_from_example(y, y_pos_tags)

    print(f'filtered {cnt_filtered} / {tot} = {cnt_filtered / tot :.2%} examples')


def joined_templates_from_examples(*args, **kwargs):
    for example, example_templates in templates_from_examples(*args, **kwargs):
        for template in example_templates:
            yield example, template


def construct_batch_by_filling_template(x, template, word_ids):
    batch_size = len(word_ids)
    x_batch = x.unsqueeze(0).expand(batch_size, *([-1] * x.dim()))
    y = torch.tensor(template.seq)
    y_batch = y.unsqueeze(0).repeat(batch_size, *([1] * y.dim()))
    y_batch[:, template.idx] = torch.tensor(word_ids)
    y_len = torch.tensor(len(template.seq))
    y_len_batch = y_len.unsqueeze(0).expand(batch_size, *([-1] * y_len.dim()))
    raw_y_batch = [[]] * batch_size
    return x_batch, y_batch, y_len_batch, raw_y_batch


def run_model_on_template(model, example, template, word_ids, batch_size=256):
    """Run model and get the whole sentence losses on template filled with every word_id in word_ids
    Inputs:
        model: the model
        example: the example; need to provide this for the model to have image x
        template: the template
        word_ids: fill the template with every word_id in word_ids
        batch_size: the batch size
    Returns:
        losses: an np.ndarray of length len(word_ids)
    """

    x, y, y_len, raw_y = example
    x = x.to(device=device)

    batches = (construct_batch_by_filling_template(x, template, word_ids[i : i + batch_size])
               for i in range(0, len(word_ids), batch_size))
    losses = []
    for batch in run_model_on_data_batches(model, batches, use_tqdm=False):
        outputs, loss = batch[4:]
        loss = loss.sum(-1)
        losses.append(loss.detach())
    losses = torch.cat(losses, 0)
    return losses


def find_index(lst, e):
    for i, x in enumerate(lst):
        if x == e:
            return i
    return None


def get_prob_dist(model, example, template, word_ids, word_cats, batch_size=256, top_k=None, words=None):
    """Run model on template and get the probability distribution of categories;
    check whether model gives higher probability to the correct category.
    Inputs:
        model: the model
        example: the example; need to provide this for the model to have image x
        template: the template
        word_ids: fill the template with every word_id in word_ids
        word_cats: np.ndarray of same length as word_ids; the categories of each word in word_ids
        batch_size: the batch size used to run the model
        top_k: print top k predictions along with the ground-truth label; None if unwanted
        words: words correspond to words_ids; used in printing
    Returns:
        pd.Series with categories indexing the probability distribution
    """

    losses = run_model_on_template(model, example, template, word_ids, batch_size=batch_size)
    probs = F.softmax(-losses, 0)
    if top_k is not None:
        used_word_ids = word_ids
        used_words = words
        used_losses = losses
        used_probs = probs
        gt_word_id = template.seq[template.idx]
        gt_word_idx = find_index(used_word_ids, gt_word_id)
        if gt_word_idx is None:  # the ground-truth word is not in the vocab of prediction
            print('ground-truth word not in the vocab of prediction')
            # append the ground-truth word to the vocab
            gt_word_idx = len(word_ids)
            used_word_ids = np.append(used_word_ids, gt_word_idx)
            used_words = np.append(used_words, idx2word[gt_word_id])
            used_losses = torch.cat([used_losses, torch.tensor([np.inf], dtype=used_losses.dtype, device=used_losses.device)])
            used_probs = torch.cat([used_probs, torch.tensor([0.], dtype=used_probs.dtype, device=used_probs.device)])
        print_top_values(-used_losses, used_words, labels=torch.tensor(gt_word_idx), value_formatter=lambda loss: f'{-loss:6.3f}')
        print_top_values(used_probs, used_words, labels=torch.tensor(gt_word_idx), value_formatter=prob_formatter)
    probs_ser = pd.Series(probs.cpu())
    return probs_ser.groupby(word_cats).sum()

In [4]:
# build data and vocab according to the model
data, args = build_data(return_args=True)
dataset_name = args.dataset
vocab = data.datasets['train'].vocab
vocab_size = len(vocab)
print(f'vocab_size = {vocab_size}')
word2idx = vocab
idx2word = [None] * vocab_size
for word, idx in word2idx.items():
    idx2word[idx] = word
idx2word = np.array(idx2word)

my_batch_size = 256
dataloader_fns = {
    'train': lambda: data.train_dataloader(batch_size=my_batch_size, shuffle=False, drop_last=False),
    'val': lambda: data.val_dataloader(batch_size=my_batch_size)[0],
    'test': lambda: data.test_dataloader(batch_size=my_batch_size)[0],
}

checkpoint_path = "checkpoints/lm_dataset_saycam_captioning_False_text_encoder_lstm_embedding_dim_512_dropout_i_0.5_dropout_o_0.0_batch_size_16_lr_0.006_lr_scheduler_True_weight_decay_0.04_seed_0/epoch=29.ckpt"
print(f"load model from {checkpoint_path}")
model = MultiModalLitModel.load_from_checkpoint(checkpoint_path, map_location=device)
model.to(device)
model.eval()

# get the pos tags of all words in vocab
# by the index of model_items produced by get_model_items
train_dataloader_fn = dataloader_fns['train']
train_pos_tags = get_pos_tags(train_dataloader_fn(), dataset_name, 'train')
train_model_items = torch_cache(checkpoint_path + f'.train.cache.pt')(get_model_items)(
    model, train_dataloader_fn(), train_pos_tags, ignore_all_token_items=True)
idx2pos = np.full(vocab_size, '.', dtype='O')
for token_id, pos in train_model_items.token_items.index:
    idx2pos[token_id] = pos
idx2pos2 = np.vectorize(pos_mapping.get)(idx2pos)
for pos in sorted(set(pos_mapping.values())):
    pos_word_ids = np.nonzero(idx2pos2 == pos)[0]
    print(f'#{pos:15s}: {len(pos_word_ids)}')
idx2used = np.isin(idx2pos2, used_poses)
if True:
    for token_id in [PAD_TOKEN_ID, SOS_TOKEN_ID, EOS_TOKEN_ID, UNK_TOKEN_ID]:
        idx2used[token_id] = False
word_ids = np.nonzero(idx2used)[0]
words = idx2word[word_ids]
word_cats = idx2pos2[word_ids]

/home/ww2135/.local/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:73: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


Using base transforms
Calling prepare_data!
SAYCam transcripts have already been downloaded. Skipping this step.
Transcripts have already been renamed. Skipping this step.
Transcripts have already been preprocessed. Skipping this step.
Training frames have already been extracted. Skipping this step.
Training metadata files have already been created. Skipping this step.
Evaluation frames have already been extracted. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Evaluation metadata files have already been created. Skipping this step.
Extra evaluation metadata files have already been created. Skipping this step.
Vocabulary file already exists. Skipping this step.
Calling setup!
vocab_size = 2350
load model from checkpoints/lm_dataset_saycam_captioning_False_text_encoder_lstm_embedding_dim_512_dropout_i_0.5_dropout_o_0.0_batch_size_16_lr_0.006_lr_scheduler_True_weight_decay_0.04_seed_0/epoch=29.ckpt
load cached pos tags: dataset_cache/saycam/t

In [5]:
splits = ['val']

my_untypical_words = ' '.join(
    possessives + negations + be_verbs + do_verbs + modal_verbs +
    pronoun_contractions + other_contractions + quantifiers + pos_ambiguous_words +
    special_tokens
).split()
filter_out_untypical_words = False

for split in splits:
    dataloader_fn = dataloader_fns[split]
    examples = examples_from_dataloader(dataloader_fn())
    pos_tags = get_pos_tags(dataloader_fn(), dataset_name, split)
    golds, preds, corrects = [], [], []
    for example, template in joined_templates_from_examples(examples, pos_tags):
        if template.seq[template.idx] in [SOS_TOKEN_ID, EOS_TOKEN_ID]:
            continue
        if filter_out_untypical_words and (idx2word[template.seq[template.idx]] in my_untypical_words):
            continue
        print(template)
        gold = pos_mapping[template.pos[template.idx]]
        prob_dist = get_prob_dist(model, example, template, word_ids, word_cats, top_k=5, words=words)
        print(prob_dist)
        argmax_idx = prob_dist.argmax()
        pred = prob_dist.index[argmax_idx]
        correct = pred == gold
        print(f'{gold = }, {pred = }, {correct = }')
        golds.append(gold)
        preds.append(pred)
        corrects.append(correct)
    print(f'label distribution: ')
    frac_format = lambda a, b, prec=2: f'{a} / {b} = {a/b:.{prec}%}'
    for pos in used_poses:
        cnt = sum(gold == pos for gold in golds)
        print(f'{pos}: {frac_format(cnt, len(golds))}')
    print(f'accuracy: {frac_format(sum(corrects), len(corrects))}')

load cached pos tags: dataset_cache/saycam/val.pos.cache
we should ____ on some lights , huh ?
28.916 turn     | 26.054 put      28.916 turn     31.390 leave    31.395 keep     31.586 get     
  5.2% turn     |  90.9% put        5.2% turn       0.4% leave      0.4% keep       0.4% get     
noun    0.003209
verb    0.996791
dtype: float32
gold = 'verb', pred = 'verb', correct = True
we should turn on some ______ , huh ?
28.916 lights   | 28.916 lights   28.961 toys     29.298 water    29.526 music    29.674 light   
 12.9% lights   |  12.9% lights    12.3% toys       8.8% water      7.0% music      6.0% light   
noun    0.980422
verb    0.019578
dtype: float32
gold = 'noun', pred = 'noun', correct = True
here , ___ me show you .
18.926 let      | 18.926 let      24.093 help     24.562 watch    26.050 lets     26.348 give    
 98.6% let      |  98.6% let        0.6% help       0.4% watch      0.1% lets       0.1% give    
noun    0.000458
verb    0.999542
dtype: float32
gold = 'verb', pr

81.940 go       | 81.940 go       89.290 are      89.907 have     90.665 do       90.783 're     
 99.8% go       |  99.8% go         0.1% are        0.0% have       0.0% do         0.0% 're     
noun    0.000055
verb    0.999946
dtype: float32
gold = 'verb', pred = 'verb', correct = True
there we go , now can you ____ me , did you want to help me pick out some clothes look we have new
81.940 help     | 79.618 show     79.951 grab     80.151 get      81.500 put      81.917 fit     
  2.8% help     |  28.3% show      20.3% grab      16.6% get        4.3% put        2.8% fit     
noun    0.020399
verb    0.979601
dtype: float32
gold = 'verb', pred = 'verb', correct = True
there we go , now can you help me , ___ you want to help me pick out some clothes look we have new
81.940 did      | 75.089 do       81.828 are      81.940 did      83.407 get      83.844 have    
  0.1% did      |  99.6% do         0.1% are        0.1% did        0.0% get        0.0% have    
noun    0.000355
verb    0

43.939 put      | 43.437 one      43.939 put      46.107 let      46.533 have     47.027 watching
 31.8% put      |  52.5% one       31.8% put        3.6% let        2.4% have       1.4% watching
noun    0.543305
verb    0.456695
dtype: float32
gold = 'verb', pred = 'noun', correct = False
i am going put more _________ on this arm , okay ?
43.939 sunscreen | 40.872 water    41.051 sand     41.161 oil      41.638 toast    41.746 books   
  0.5% sunscreen |  10.3% water      8.6% sand       7.7% oil        4.8% toast      4.3% books   
noun    0.933721
verb    0.066279
dtype: float32
gold = 'noun', pred = 'noun', correct = True
i am going put more sunscreen on this ___ , okay ?
43.939 arm      | 41.430 time     41.484 way      41.806 foot     42.109 one      42.905 cake    
  1.1% arm      |  13.8% time      13.0% way        9.4% foot       7.0% one        3.1% cake    
noun    0.971862
verb    0.028138
dtype: float32
gold = 'noun', pred = 'noun', correct = True
which ___ do you want ?
 

ground-truth word not in the vocab of prediction
   inf like     | 23.790 see      25.435 remember 25.530 hear     25.674 know     26.216 want    
  0.0% like     |  50.4% see        9.7% remember   8.8% hear       7.7% know       4.5% want    
noun    0.003377
verb    0.996623
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you like the the ______ ?
23.044 magpie   | 17.441 ball     18.004 kitty    18.379 cup      18.635 car      18.745 cow     
  0.0% magpie   |  10.8% ball       6.2% kitty      4.2% cup        3.3% car        2.9% cow     
noun    0.981424
verb    0.018576
dtype: float32
gold = 'noun', pred = 'noun', correct = True
op that __ not exactly how it is .
28.220 's       | 28.220 's       30.588 s        31.491 ’s       33.006 is       35.014 're     
 87.5% 's       |  87.5% 's         8.2% s          3.3% ’s         0.7% is         0.1% 're     
noun    0.000014
verb    0.999987
dtype: float32
gold = 'verb', pred = 'verb', correct = True
op that 's not exact

16.672 do       | 16.672 do       21.019 did      22.861 are      25.336 remember 25.823 thank   
 98.4% do       |  98.4% do         1.3% did        0.2% are        0.0% remember   0.0% thank   
noun    0.000073
verb    0.999927
dtype: float32
gold = 'verb', pred = 'verb', correct = True
" do you ________ how to turn it on ?
16.672 remember | 16.672 remember 18.830 know     21.400 want     22.183 remeber  22.240 see     
 88.2% remember |  88.2% remember  10.2% know       0.8% want       0.4% remeber    0.3% see     
noun    0.000027
verb    0.999973
dtype: float32
gold = 'verb', pred = 'verb', correct = True
" do you remember how to ____ it on ?
16.672 turn     | 16.672 turn     20.528 put      22.325 get      22.620 make     23.250 keep    
 96.4% turn     |  96.4% turn       2.0% put        0.3% get        0.3% make       0.1% keep    
noun    0.000594
verb    0.999406
dtype: float32
gold = 'verb', pred = 'verb', correct = True
but , sometimes i ______ and slip and sprawl . "
25.86

...

21.852 ohp      | 15.581 see      16.274 look     18.439 remember 19.279 sam      19.872 kitty   
  0.1% ohp      |  56.7% see       28.4% look       3.3% remember   1.4% sam        0.8% kitty   
noun    0.093603
verb    0.906397
dtype: float32
gold = 'noun', pred = 'verb', correct = False
ohp , he __ gone .
21.852 is       | 20.562 's       21.852 is       22.856 was      22.924 s        23.737 gets    
 17.6% is       |  64.1% 's        17.6% is         6.5% was        6.0% s          2.7% gets    
noun    0.000509
verb    0.999491
dtype: float32
gold = 'verb', pred = 'verb', correct = True
ohp , he is ____ .
21.852 gone     | 18.806 twisting 20.241 eating   21.753 painting 21.753 laughing 21.852 gone    
  2.5% gone     |  52.4% twisting  12.5% eating     2.7% painting   2.7% laughing   2.5% gone    
noun    0.12732
verb    0.87268
dtype: float32
gold = 'verb', pred = 'verb', correct = True
____ some grapefruit ?
15.466 want     | 15.466 want     17.611 try      19.099 get      19.2

44.811 look     | 44.811 look     46.749 play     46.974 looking  48.433 typing   49.034 do      
 64.8% look     |  64.8% look       9.3% play       7.5% looking    1.7% typing     1.0% do      
noun    0.03441
verb    0.96559
dtype: float32
gold = 'verb', pred = 'verb', correct = True
why do nt you look at ______ dog instead .
44.811 mister   | 42.549 stuffed  43.346 rice     43.549 marmite  44.046 music    44.169 daycare 
  1.3% mister   |  12.9% stuffed    5.8% rice       4.8% marmite    2.9% music      2.6% daycare 
noun    0.648609
verb    0.351391
dtype: float32
gold = 'noun', pred = 'noun', correct = True
why do nt you look at mister ___ instead .
44.811 dog      | 39.204 one      40.537 fly      40.573 kitty    40.856 things   41.111 car     
  0.1% dog      |  17.3% one        4.6% fly        4.4% kitty      3.3% things     2.6% car     
noun    0.914504
verb    0.085496
dtype: float32
gold = 'noun', pred = 'noun', correct = True
oh that __ so pretty , look at that
29.317 's 

16.810 baby     | 16.810 baby     20.730 girl     21.850 one      21.961 kitty    22.435 bear    
 95.4% baby     |  95.4% baby       1.9% girl       0.6% one        0.6% kitty      0.3% bear    
noun    0.999801
verb    0.000199
dtype: float32
gold = 'noun', pred = 'noun', correct = True
there s a happy baby and a ____ with blocks .
16.810 baby     | 16.810 baby     21.264 girl     22.953 boy      23.175 one      23.404 bear    
 97.8% baby     |  97.8% baby       1.1% girl       0.2% boy        0.2% one        0.1% bear    
noun    0.999877
verb    0.000123
dtype: float32
gold = 'noun', pred = 'noun', correct = True
there s a happy baby and a baby with ______ .
16.810 blocks   | 16.810 blocks   19.910 flowers  20.512 swimming 21.016 eyes     21.118 bubbles 
 82.8% blocks   |  82.8% blocks     3.7% flowers    2.0% swimming   1.2% eyes       1.1% bubbles 
noun    0.968032
verb    0.031968
dtype: float32
gold = 'noun', pred = 'noun', correct = True
it __ okay .
 7.917 's       |  7.917 

25.387 mouth    | 22.387 foot     24.615 hand     25.387 mouth    25.763 water    25.780 coming  
  2.9% mouth    |  59.1% foot       6.4% hand       2.9% mouth      2.0% water      2.0% coming  
noun    0.947291
verb    0.052709
dtype: float32
gold = 'noun', pred = 'noun', correct = True
you ____ to put those in the bin ?
15.839 want     | 15.839 want     17.940 going    19.579 need     19.666 have     21.372 trying  
 84.5% want     |  84.5% want      10.3% going      2.0% need       1.8% have       0.3% trying  
noun    0.000313
verb    0.999687
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you want to ___ those in the bin ?
15.839 put      | 15.839 put      19.404 throw    19.707 get      19.828 bring    20.323 see     
 87.2% put      |  87.2% put        2.5% throw      1.8% get        1.6% bring      1.0% see     
noun    0.004179
verb    0.995821
dtype: float32
gold = 'verb', pred = 'verb', correct = True
you want to put those in the ___ ?
15.839 bin      | 15.839 

33.708 one      | 33.364 sock     33.708 one      34.828 hand     35.196 way      35.521 bit     
 15.2% one      |  21.5% sock      15.2% one        5.0% hand       3.4% way        2.5% bit     
noun    0.94159
verb    0.05841
dtype: float32
gold = 'noun', pred = 'noun', correct = True
i 'm gon na have this one , can you ___ it in the bowl ?
33.708 put      | 33.708 put      36.462 pour     38.495 pick     39.085 stick    39.230 pull    
 88.3% put      |  88.3% put        5.6% pour       0.7% pick       0.4% stick      0.4% pull    
noun    0.005753
verb    0.994247
dtype: float32
gold = 'verb', pred = 'verb', correct = True
i 'm gon na have this one , can you put it in the ____ ?
33.708 bowl     | 29.469 bin      30.528 bag      30.929 backpack 31.377 cup      31.784 car     
  0.5% bowl     |  37.0% bin       12.9% bag        8.6% backpack   5.5% cup        3.7% car     
noun    0.993641
verb    0.006359
dtype: float32
gold = 'noun', pred = 'noun', correct = True
that __ right
 6.6